In [1]:
import pandas as pd
import numpy as np
import ast
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import re
from tqdm.notebook import tqdm

In [2]:
# !pip install selenium

In [3]:
train = pd.read_csv('train_dataset_train.csv')
test = pd.read_csv('test_dataset_test.csv')
data = pd.concat([train, test])
ltr = len(train)
data = data.reset_index(drop = True)
data['tags'] = data['tags'].apply(ast.literal_eval)

In [4]:
data = pd.concat([train, test])
ltr = len(train)

In [5]:
data = data.reset_index(drop = True)

In [6]:
data['tags'] = data['tags'].apply(ast.literal_eval)
data['authors'] = data['authors'].apply(ast.literal_eval)

In [7]:
cnt = Counter(np.concatenate(data['authors'].tolist()))
data['vc_authors'] = [sum([cnt[x] for x in a]) for a in data['authors']]

In [8]:
cnt = Counter(np.concatenate(data['tags'].tolist()))
data['vc_tags'] = [sum([cnt[x] for x in a]) for a in data['tags']]

In [9]:
data['len_tags'] = [len(a) for a in data['tags']]

In [10]:
import pickle
with open('list_data_new_1.pickle', 'rb') as f:
    text_data = pickle.load(f)

In [11]:
data['site'] = [x[0] for x in text_data]
data['overview'] = [x[2] for x in text_data]
data['photo'] = [x[3] for x in text_data]

In [12]:
data['header'] = [x[1].split(',')[0] for x in text_data]

In [13]:
data['full_text'] = [' '.join(x[-1][:[i for i, x in enumerate(x[-1]) if x == ''][0]]) 
    if len( [i for i, x in enumerate(x[-1]) if x == '']) else ' '.join(x[-1]) for x in text_data]

In [14]:
data['full_text_d'] = [' '.join(x[4]) for x in text_data]

In [15]:
data['first_sent_text_d'] = [x[4][0] if len(x[4]) else '' for x in text_data]

In [16]:
data['publish_date'] = pd.to_datetime(data['publish_date'])

In [17]:
data['feat_days'] = (data['publish_date'] - pd.to_datetime('2017-01-01') ).dt.days

In [18]:
data['hours'] = data['publish_date'].dt.hour * 60 + data['publish_date'].dt.minute
data['weekday'] = data['publish_date'].dt.weekday

In [19]:
data['day'] = data['publish_date'].dt.day

In [20]:
data['clean_title'] = [x.split('\n')[0] for x in data['title']]

In [21]:
data['vc_header'] = data['header'].map(data['header'].value_counts())

In [22]:
data['len_clean_title'] = data['clean_title'].apply(lambda x:len(x))
data['len_clean_title_d'] = data['title'].apply(lambda x:len(x))
data['len_full_text'] = data['full_text'].apply(lambda x:len(x))
data['len_full_text_d'] = data['full_text_d'].apply(lambda x:len(x))

data['len_title'] = data['title'].apply(lambda x:len(x))
data['len_authors'] = data['authors'].apply(lambda x:len(x))
data['len_tags'] = data['tags'].apply(lambda x:len(x))
data['len_photo'] = data['photo'].apply(lambda x:len(x))
data['len_overview'] = data['overview'].apply(lambda x:len(x))
data['all_len'] = data['len_clean_title'] + data['len_overview']
data['len_numbers_title'] = [len(re.findall('\d+', x)) for x in data['clean_title']]
data['len_english_title'] = [len(re.findall('[A-z]', x)) for x in data['clean_title']]
data['len_dots_title'] = [len(x.split('. ')) for x in data['clean_title']]
data['len_english_text'] = [len(re.findall('[A-z]', x)) for x in data['full_text']]
data['len_numbers_text'] = [len(re.findall('\d+', x)) for x in data['full_text']]
data['len_dots_text'] = [len(x.split('. ')) for x in data['full_text']]
data['len_overview'] = data['overview'].apply(lambda x:len(x))
data['len_sites_split'] = [len(x.split('/')) for x in data['site']]

data['len_parse'] = [len(x[4]) for x in text_data]

data['abzac_full_text'] = [len( [i for i, x in enumerate(x[-1]) if x == '']) for x in text_data]
data['abzac_full_text0'] = [np.std( [len(x) for i, x in enumerate(x[-1]) if x != '']) for x in text_data]
# session

C:\Users\Sergei\anaconda3\lib\site-packages\numpy\core\_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Sergei\anaconda3\lib\site-packages\numpy\core\_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Sergei\anaconda3\lib\site-packages\numpy\core\_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [23]:
tfidf = TfidfVectorizer(max_features = 1000)
tfidf_data = tfidf.fit_transform(data['clean_title'] + ' ' + data['overview'])
data_tfidf = pd.DataFrame(tfidf_data.toarray())
data_tfidf.columns = [f'tfidf_title_{x}' for x in data_tfidf.columns]

In [24]:
# tfidf_fs = TfidfVectorizer(max_features = 1000)
# tfidf_data_fs = tfidf_fs.fit_transform(data['first_sent_text_d'])
# data_tfidf_fs = pd.DataFrame(tfidf_data_fs.toarray())
# data_tfidf_fs.columns = [f'tfidf_title_fs_{x}' for x in data_tfidf_fs.columns]

In [25]:
tfidf_text = TfidfVectorizer(max_features = 5000)
tfidf_data_text = tfidf_text.fit_transform(data['full_text'])
data_text_tfidf = pd.DataFrame(tfidf_data_text.toarray())
data_text_tfidf.columns = [f'tfidf_text_{x}' for x in data_text_tfidf.columns]

tfidf_text_d = TfidfVectorizer(max_features = 5000)
tfidf_data_d_text = tfidf_text_d.fit_transform(data['full_text_d'])
data_text_tfidf_d = pd.DataFrame(tfidf_data_d_text.toarray())
data_text_tfidf_d.columns = [f'tfidf_text_d_{x}' for x in data_text_tfidf_d.columns]

In [26]:
data['vc_category'] = data['category'].map(data['category'].value_counts())

In [27]:
tfidf_tags = TfidfVectorizer(max_features = 100)
tfidf_tags_data = tfidf_tags.fit_transform(data['tags'].apply(lambda x: ' '.join(x)))
data_tags_tfidf = pd.DataFrame(tfidf_tags_data.toarray())
data_tags_tfidf.columns = [f'tfidf_tags_{x}' for x in data_tags_tfidf.columns]

In [28]:
tfidf_aut = TfidfVectorizer(min_df = 10)
tfidf_aut_data = tfidf_aut.fit_transform(data['authors'].apply(lambda x: ' '.join(x)))
data_aut_tfidf = pd.DataFrame(tfidf_aut_data.toarray())
data_aut_tfidf.columns = [f'tfidf_aut_{x}' for x in data_aut_tfidf.columns]

In [29]:
tfidf_photo = TfidfVectorizer(min_df = 20)
tfidf_photo_text = tfidf_photo.fit_transform(data['feat_days'].astype('str'))
data_photo_tfidf = pd.DataFrame(tfidf_photo_text.toarray())
data_photo_tfidf.columns = [f'tfidf_photo_{x}' for x in data_photo_tfidf.columns]

In [30]:
tfidf_head = TfidfVectorizer(min_df = 10)
tfidf_head_data = tfidf_head.fit_transform(data['header'])
data_head_tfidf = pd.DataFrame(tfidf_head_data.toarray())
data_head_tfidf.columns = [f'tfidf_head_{x}' for x in data_head_tfidf.columns]

In [31]:
tfidf_cat = TfidfVectorizer(min_df = 10)
tfidf_cat_data = tfidf_cat.fit_transform(data['category'])
data_cat_tfidf = pd.DataFrame(tfidf_cat_data.toarray())
data_cat_tfidf.columns = [f'tfidf_cat_{x}' for x in data_cat_tfidf.columns]

In [32]:
from sklearn.decomposition import TruncatedSVD 
pca = TruncatedSVD(10, random_state = 121)
pca_cat_data = pca.fit_transform(tfidf_data)
data_cat_pca = pd.DataFrame(pca_cat_data)
data_cat_pca.columns = [f'pca_cat_{x}' for x in data_cat_pca.columns]

pca = TruncatedSVD(15, random_state = 121)
pca_cat_data = pca.fit_transform(tfidf_tags_data)
data_fs_pca = pd.DataFrame(pca_cat_data)
data_fs_pca.columns = [f'pca_fs_{x}' for x in data_fs_pca.columns]

pca = TruncatedSVD(20, random_state = 121)
# pca = TruncatedSVD(20, random_state = 311)
pca_text_data = pca.fit_transform(tfidf_data_text)
data_text_pca = pd.DataFrame(pca_text_data)
data_text_pca.columns = [f'pca_text_{x}' for x in data_text_pca.columns]

pca = TruncatedSVD(25, random_state = 121)
# pca = TruncatedSVD(20, random_state = 311)
pca_text_data_d = pca.fit_transform(tfidf_data_d_text)
data_text_pca_d = pd.DataFrame(pca_text_data_d)
data_text_pca_d.columns = [f'pca_textd_{x}' for x in data_text_pca_d.columns]

In [33]:
data = pd.concat([data, data_tfidf, data_tags_tfidf, data_text_tfidf, data_aut_tfidf, data_fs_pca, data_head_tfidf, data_cat_tfidf, data_cat_pca, data_text_pca, data_text_pca_d, data_photo_tfidf], 1)

C:\Users\Sergei\AppData\Local\Temp/ipykernel_3084/2904110049.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  data = pd.concat([data, data_tfidf, data_tags_tfidf, data_text_tfidf, data_aut_tfidf, data_fs_pca, data_head_tfidf, data_cat_tfidf, data_cat_pca, data_text_pca, data_text_pca_d, data_photo_tfidf], 1)


In [34]:
data['svo'] = [1 if 'видео' in x.lower() else 0 for x in data['full_text'] ]
data['dates'] = data.publish_date.dt.date
data['vc_dates'] = data['dates'].map(data['dates'].value_counts())
data['ctr_mean'] = data['dates'].map( data[data.ctr != 0].groupby('dates').ctr.mean() )
data['ctr_diff'] = data['ctr'] - data['ctr_mean']

data['link'] = [x[0][:-len(x[1])] for x in data[['document_id', 'session']].values]
data['tmp1'] = [len(re.findall('AP', x)) for x in data['photo']]
data['tmp2'] = [len(re.findall('Instagrams', x)) for x in data['photo']]

data['abzac_full_text2'] = [np.max( [len(x) for i, x in enumerate(x[-1]) ]) if len(x[-1]) else 0 for x in text_data]
data['abzac_full_text3'] = [len(x[-1][0]) if len(x[-1]) else 0  for x in text_data]
data['abzac_full_text4'] = [len(x[-1][1]) if len(x[-1]) > 1 else 0  for x in text_data]
# data['abzac_full_text4'] = data['abzac_full_text3'] - data['len_parse']

data['len_diff1'] = data['len_clean_title'] / data['len_clean_title_d']
data['len_diff2'] = data['len_dots_text'] / data['len_full_text']
data['len_diff3'] = data['len_dots_title'] / data['len_clean_title']
data['len_diff4'] = data['len_full_text'] / data['len_full_text_d']
data['len_diff5'] = data['len_full_text'] - data['len_full_text_d']
data['len_diff6'] = data['len_title'] / data['len_overview']

dict_mean_ctr = {}
for ctr, a_l in data[['ctr', 'tags']].values:
    if ctr != 0:
        for a in a_l:
            dict_mean_ctr[a] = dict_mean_ctr.get(a, []) + [ctr]
data['ctr_mean_tags'] = [np.mean([np.mean(dict_mean_ctr.get(x, [1])) for x in a]) for a in data['tags']]

dict_mean_ctr = {}
for ctr, a_l in data[['ctr', 'authors']].values:
    if ctr != 0:
        for a in a_l:
            dict_mean_ctr[a] = dict_mean_ctr.get(a, []) + [ctr]
data['ctr_mean_authors'] = [np.mean([np.mean(dict_mean_ctr.get(x, [1])) for x in a]) for a in data['authors']]

data['dum1'] = data[[col for col in data.columns if 'tfidf_title_' in col]].sum(1)
data['dum2'] = data[[col for col in data.columns if 'tfidf_head_' in col]].sum(1)
data['dum3'] = data[[col for col in data.columns if 'tfidf_aut_' in col]].sum(1)

C:\Users\Sergei\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Sergei\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [35]:
with open('list_data_new_full_test.pickle', 'rb') as f:
    full_text_data = pickle.load(f)
    
feat_data = []
for x in full_text_data:
    wer = [i[0] for i in x[1]]
    tmp_ind = []
    tmp = []
    for i, w in enumerate(wer):
        if w == '':
            if len(tmp):
                tmp_ind += [tmp]
            tmp = []
        else:
            tmp += [i]
    if len(tmp):
        tmp_ind += [tmp]
    lens = [np.sum([x[1][j][1][0] for j in i]) for i in tmp_ind]
    if lens:
        m = np.max(lens)
    else:
        m = -1
    l = len(lens)
    lens = lens[:4]
    lens += [-1] * (4 - len(lens))
    feat_data += [lens + [l,m]]
    
feat_data = np.array(feat_data)
for i in range(6):
    data[f'nre_{i}'] = feat_data[:, i]
data.loc[:, 'new1'] = [x[0][1][1][0] for x in full_text_data]
data.loc[:, 'new2'] = [x[0][0][1][0] for x in full_text_data]
data.loc[:, 'new3'] = [x[0][2][1][0] for x in full_text_data]
data.loc[:, 'new4'] = [x[0][3][1][0] for x in full_text_data]
data.loc[:, 'new5'] = [x[0][4][1][0] for x in full_text_data]
data.loc[:, 'new6'] = [x[0][5][1][0] for x in full_text_data]
data.loc[:, 'new7'] = [x[0][6][1][0] for x in full_text_data]
data.loc[:, 'new8'] = [x[0][7][1][0] for x in full_text_data]
data.loc[:, 'new9'] = [x[0][8][1][0] for x in full_text_data]
data.loc[:, 'new10'] = [x[0][9][1][0] for x in full_text_data]
data.loc[:, 'new12'] = [x[1][0][1][0] if len(x[1]) else -1 for x in full_text_data]
data.loc[:, 'new11'] = data['new12'] / data['new1']


data.loc[:, 'lnew1'] = [len(x[0][0][0]) for x in full_text_data]
data.loc[:, 'lnew2'] = [len(x[0][1][0]) for x in full_text_data]
data.loc[:, 'lnew3'] = [len(x[0][2][0]) for x in full_text_data]
data.loc[:, 'lnew4'] = [len(x[0][3][0]) for x in full_text_data]
data.loc[:, 'lnew5'] = [len(x[0][4][0]) for x in full_text_data]
data.loc[:, 'lnew6'] = [len(x[0][5][0]) for x in full_text_data]
data.loc[:, 'lnew7'] =[len(x[0][6][0]) for x in full_text_data]
data.loc[:, 'lnew8'] = [len(x[0][7][0]) for x in full_text_data]
data.loc[:, 'lnew9'] = [len(x[0][8][0]) for x in full_text_data]
data.loc[:, 'lnew10'] = [len(x[0][9][0]) for x in full_text_data]


data.loc[:, 'compy1'] = [(np.array([i[1][0] for i in x[1] if 'фото' not in i[0].lower()])).sum() for x in full_text_data]
data.loc[:, 'compy2'] = [((np.array([i[1][0] for i in x[1]])) == 0).sum() for x in full_text_data]
data['compy3'] = data['compy1'] / data['new1']

data.loc[: ,'compy4'] = [(np.array([i[1][0] for i in x[1] ])).sum() for x in full_text_data]
data.loc[:, 'compy5'] = [(np.array([i[1][0] for i in x[1] if 'фото' in i[0].lower()])).sum() for x in full_text_data]

data['compy6'] = data['compy5'] / data['new1']
data['compy7'] = data['compy4'] / data['new1']
data['compy8'] = data['compy2'] / data['new1']

data['num_abzac'] = feat_data[:, -1]
data['compy'] = [len(re.findall( '\\n', x[0][1][0] )) for x in full_text_data]
data['diff1'] = data['len_overview'] / data['len_full_text'] 
data['diff2'] = feat_data[:, 0] / data['new1'] 
data['diff3'] = data['compy'] / data['len_full_text'] 
data['dr'] = data['new1'] / data['len_full_text']

In [36]:
def lgb_train(data, target, ltr, split_list, param, v_e = 0, n_e = 5000, cat_col = None):
    pred = pd.DataFrame()
    pred_val = np.zeros(len(data))
    fi = np.zeros(data.shape[1])
    score = []
    bst_list = []
    pred_train = pd.DataFrame()
    j = 0
    target = target 
     
    full_tr =lgb.Dataset(np.array(data)[:ltr],np.array(target)[:ltr])
    dict_score = lgb.cv(param, full_tr, 5000, split_list, early_stopping_rounds=300, verbose_eval = 100)
    best_tree = np.argmin(dict_score['l2-mean']) + 1
    print(np.min(dict_score['l2-mean']))
#     best_tree = 768
    for i , (train_index, test_index) in enumerate(split_list):
#         drop_index = data[]
        tr = lgb.Dataset(np.array(data)[train_index],np.array(target)[train_index] , reference=full_tr)
        te = lgb.Dataset(np.array(data)[test_index], np.array(target)[test_index], reference=tr)
        evallist = [(tr, 'train'), (te, 'test')]
        
#         bst = lgb.train(param, tr, num_boost_round=n_e, 
#                 valid_sets = [tr, te], early_stopping_rounds=int(5 / param['learning_rate']), verbose_eval = v_e)
        bst = lgb.train(param, tr, num_boost_round=best_tree)
#         return bst
        pred[str(i)] = bst.predict(np.array(data)[ltr:, :])
        pred_val[test_index] = bst.predict(np.array(data)[test_index]) 
        score += [r2_score(np.array(target)[test_index] , pred_val[test_index])]
        bst_list += [bst]
        fi += np.array(bst.feature_importance(importance_type = 'gain'))
        print(i+1, np.mean(score))
        print()
        del tr, te
    pred_train[str(j)] = pred_val
    return bst_list, pred_train, pred, score, fi

In [37]:
ltr = len(train)

In [38]:
tmp = data['link'] + '_' + data['ctr'].astype('str')
tmp_vc = tmp.value_counts()
have_indexs = tmp[tmp.isin(tmp_vc[tmp_vc >= 2].index)].index

In [40]:
from sklearn.model_selection import StratifiedKFold, KFold
def standart_split(data, ltr):
    split_list = []
    kf = KFold(n_splits = 10, shuffle = True, random_state = 228)
    for train_index, test_index in kf.split(data.loc[:ltr-1, :], data.loc[:ltr-1, 'views']):
        test_index = sorted(set(test_index) - set(have_indexs))
        split_list += [(train_index, test_index)]

    return split_list
split_list = standart_split(data, ltr)

In [41]:
# data[data[ 'publish_date'] > '2021-01-01']

In [42]:
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error

In [43]:
# np.mean(score_1) * 0.4 + np.mean(score_2) * 0.3 +  + np.mean(score_3) * 0.3

In [44]:
param_lgb_view = {
# 'bagging_fraction': 0.9,
# 'bagging_freq': 1,
'boost': 'gbdt',
#     'max_bin' : 500,
#     'linear_lambda' : 1,
# 'feature_fraction': 1,
'learning_rate': 0.1,
'metric':'l2',
'num_leaves': 12,
        'verbosity':-1,
'objective': 'l2',
        'lambda_l2':1,
    'reg_sqrt':True
}

In [46]:
train_cols_view = ['ctr', 'vc_authors', 'vc_tags', 'feat_days', 'hours',  'day', 'vc_header', 
                   'len_clean_title_d', 'len_full_text', 'len_authors', 'len_tags',  'len_overview',
                   'all_len', 'len_numbers_title', 'len_dots_title', 'len_english_text', 
                   'len_numbers_text', 'len_dots_text', 'len_sites_split', 'len_diff1', 
                   'len_parse', 'abzac_full_text', 'abzac_full_text0', 'vc_category', 'svo', 'vc_dates', 'ctr_mean',
                    'tmp1',   'nre_5', 'abzac_full_text4', 'lnew5', 'new5', 'dum2', 'compy8',
                   'lnew1', 'new10', 'new9', 'compy1', 'compy7', 'diff3'   ]


train_cols_view += [x for x in data.columns if 'tfidf_cat_' in x] 
# train_cols_view += [x for x in data.columns if 'tfidf_head_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_aut_' in x] 
# train_cols_view += [x for x in data.columns if 'pca_cat_' in x]
train_cols_view += [x for x in data.columns if 'pca_textd_' in x]
train_cols_view += [x for x in data.columns if 'pca_text_' in x]
# train_cols_view += ['svo1']
bst_list, pred_train1, pred_views1, score_1, fi = lgb_train(data[train_cols_view], data['views'], ltr, split_list, param_lgb_view, 50)

C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's l2: 1.59414e+09 + 1.8868e+09
[200]	cv_agg's l2: 1.56794e+09 + 1.81787e+09
[300]	cv_agg's l2: 1.56586e+09 + 1.8019e+09
[400]	cv_agg's l2: 1.55881e+09 + 1.78464e+09
[500]	cv_agg's l2: 1.5636e+09 + 1.78497e+09
[600]	cv_agg's l2: 1.56164e+09 + 1.77243e+09
[700]	cv_agg's l2: 1.56482e+09 + 1.77303e+09
1557197835.2419212
1 0.628327480252117

2 0.6268876589612847

3 0.5659862266275726

4 0.5809379635128021

5 0.5766657065384966

6 0.5582551274924726

7 0.5500023740581373

8 0.561061097865258

9 0.5669770024000025

10 0.557229334016876



In [47]:
train_cols_view = ['ctr', 'vc_authors', 'vc_tags', 'feat_days', 'hours', 'weekday', 'day', 'vc_header', 'len_clean_title',
                   'len_clean_title_d', 'len_full_text', 'len_authors', 'len_tags', 'len_photo', 'len_overview',
                   'all_len', 'len_numbers_title', 'len_dots_title', 'len_english_text', 
                   'len_numbers_text', 'len_dots_text', 'len_sites_split', 'len_diff1', 
                   'len_parse', 'abzac_full_text', 'abzac_full_text0', 'vc_category', 'svo', 'vc_dates', 'ctr_mean',
                   'ctr_diff', 'tmp1',     ]


train_cols_view += [x for x in data.columns if 'tfidf_cat_' in x] 
# train_cols_view += [x for x in data.columns if 'tfidf_head_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_aut_' in x] 
# train_cols_view += [x for x in data.columns if 'pca_cat_' in x]
train_cols_view += [x for x in data.columns if 'pca_textd_' in x]
train_cols_view += [x for x in data.columns if 'pca_text_' in x]
# train_cols_view += ['svo1']
bst_list, pred_train2, pred_views2, score_1, fi = lgb_train(data[train_cols_view], data['views'], ltr, split_list, param_lgb_view, 50)

C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's l2: 1.57226e+09 + 1.86785e+09
[200]	cv_agg's l2: 1.52142e+09 + 1.76545e+09
[300]	cv_agg's l2: 1.51441e+09 + 1.73639e+09
[400]	cv_agg's l2: 1.50986e+09 + 1.71975e+09
[500]	cv_agg's l2: 1.51007e+09 + 1.70789e+09
[600]	cv_agg's l2: 1.50802e+09 + 1.69773e+09
[700]	cv_agg's l2: 1.5072e+09 + 1.69086e+09
[800]	cv_agg's l2: 1.50031e+09 + 1.66799e+09
[900]	cv_agg's l2: 1.49915e+09 + 1.66347e+09
[1000]	cv_agg's l2: 1.50161e+09 + 1.6666e+09
[1100]	cv_agg's l2: 1.50346e+09 + 1.67239e+09
1498281738.2814255
1 0.6315043431797966

2 0.6251725983031924

3 0.5761676940936707

4 0.593729710658726

5 0.5921085520591433

6 0.5730305376724896

7 0.5703987387899493

8 0.5828593383151872

9 0.5886586987831975

10 0.5691426495242544



In [49]:
comp_ind = sorted(set(range(7000)) - set(have_indexs))
data['pred'] = pred_train2['0']
data.loc[(data.ctr >= 0) , 'pred'] *= 1.15
mean_squared_error(data['views'][comp_ind], data['pred'][comp_ind]) / 1e9
r2_score(data['views'][comp_ind].tolist() + data['views'][have_indexs[have_indexs < 7000]].tolist(),
        data['pred'][comp_ind].tolist() +  data['views'][have_indexs[have_indexs < 7000]].tolist()) 

0.8422580971698007

In [51]:
param_lgb_view = {
# 'bagging_fraction': 0.9,
# 'bagging_freq': 1,
'boost': 'gbdt',
#     'max_bin' : 100,
#     'linear_lambda' : 1,
# 'feature_fraction': 1,
'learning_rate': 0.1,
'metric':'l2',
'num_leaves': 12,
        'verbosity':-1,
'objective': 'l2',
        'lambda_l2':1,
    'reg_sqrt':True
}


In [52]:
train_cols_view = ['ctr', 'vc_authors', 'vc_tags', 'feat_days', 'hours', 'weekday', 'day', 'vc_header', 'len_clean_title',
                   'len_full_text', 'len_authors', 'len_tags', 'len_overview',
                   'all_len', 'len_dots_text', 'len_sites_split', 'compy', 'compy1', 'compy3', 
                   'len_parse', 'abzac_full_text', 'abzac_full_text0', 'vc_category','vc_dates', 'ctr_mean',
                   'ctr_diff', 'abzac_full_text2', 'abzac_full_text3', 'len_diff5', 
                  'len_diff2', 'len_full_text_d',  'len_diff3', 'len_diff4', 'len_diff6', 'len_title',
                  'ctr_mean_tags', 'new1',  'new11', 'lnew10', 'new6', 'compy2'                 ]

train_cols_view += [x for x in data.columns if 'tfidf_cat_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_tags_' in x] 
# train_cols_view += [x for x in data.columns if 'tfidf_photo_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_aut_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_head_' in x] 
# train_cols_view += [x for x in data.columns if 'pca_cat_' in x]
train_cols_view += [x for x in data.columns if 'pca_textd_' in x]
# train_cols_view += [x for x in data.columns if 'pca_text_' in x]
bst_list, pred_train_c1, pred_ftr1, score_1, fi = lgb_train(data[train_cols_view], data['full_reads_percent'], ltr, split_list, param_lgb_view, 50)

C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's l2: 54.1591 + 29.7559
[200]	cv_agg's l2: 53.4609 + 29.8557
[300]	cv_agg's l2: 53.3588 + 30.006
[400]	cv_agg's l2: 53.3415 + 29.9476
[500]	cv_agg's l2: 53.5366 + 30.0527
[600]	cv_agg's l2: 53.7078 + 30.1013
53.267250105549046
1 0.6150626495623017

2 0.616905085395435

3 0.6085259809091391

4 0.5995251665374224

5 0.605419851613795

6 0.5597375083845898

7 0.5738678762033386

8 0.5763955586297727

9 0.5496420831284127

10 0.5585570756537601



In [53]:
train_cols_view = ['ctr', 'vc_authors', 'vc_tags', 'feat_days', 'hours', 'weekday', 'day', 'vc_header', 'len_clean_title',
                   'len_full_text', 'len_authors', 'len_tags', 'len_overview',
                   'all_len', 'len_dots_text', 'len_sites_split',
                   'len_parse', 'abzac_full_text', 'abzac_full_text0', 'vc_category','vc_dates', 'ctr_mean',
                   'ctr_diff', 'abzac_full_text2', 'abzac_full_text3', 'len_diff5',
                  'len_diff2', 'len_full_text_d',  'len_diff3', 'len_diff4', 'len_diff6', 'len_title',
                  'ctr_mean_tags']

train_cols_view += [x for x in data.columns if 'tfidf_cat_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_tags_' in x] 
# train_cols_view += [x for x in data.columns if 'tfidf_photo_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_aut_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_head_' in x] 
# train_cols_view += [x for x in data.columns if 'pca_cat_' in x]
train_cols_view += [x for x in data.columns if 'pca_textd_' in x]
train_cols_view += [x for x in data.columns if 'pca_text_' in x]
bst_list, pred_train_c2, pred_ftr2, score_1, fi = lgb_train(data[train_cols_view], data['full_reads_percent'], ltr, split_list, param_lgb_view, 50)

C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's l2: 54.9609 + 29.4174
[200]	cv_agg's l2: 54.4599 + 29.5317
[300]	cv_agg's l2: 54.3963 + 29.4612
[400]	cv_agg's l2: 54.5199 + 29.9217
[500]	cv_agg's l2: 54.7263 + 30.0142
54.357319081124615
1 0.6272408172021955

2 0.6102544532286869

3 0.6030946416982798

4 0.5928969262996571

5 0.5945018850185886

6 0.5507479288655468

7 0.5622566005410014

8 0.564465914671801

9 0.5390426672512186

10 0.5478156290706107



In [54]:
comp_ind = sorted(set(range(7000)) - set(have_indexs))
data['pred'] =( pred_train_c1['0'] + pred_train_c2['0']) /2
data.loc[(data.ctr > 0) , 'pred'] *= 1
# mean_squared_error(data['full_reads_percent'][comp_ind], data['pred'][comp_ind]) / 1e9
r2_score(data['full_reads_percent'][comp_ind].tolist() + data['full_reads_percent'][have_indexs[have_indexs < 7000]].tolist(),
        data['pred'][comp_ind].tolist() +  data['full_reads_percent'][have_indexs[have_indexs < 7000]].tolist()) 

0.5521421364634166

In [57]:
from sklearn.model_selection import StratifiedKFold, KFold
def standart_split(data, ltr):
    split_list = []
    kf = KFold(n_splits = 10, shuffle = True, random_state = 228)
    for train_index, test_index in kf.split(data.loc[:ltr-1, :], data.loc[:ltr-1, 'views']):
        train_index = sorted(set(train_index) - set(have_indexs))
        test_index = sorted(set(test_index) - set(have_indexs))
        split_list += [(train_index, test_index)]

    return split_list
split_list_new = standart_split(data, ltr)

In [58]:
param_lgb = {
# 'bagging_fraction': 0.9,
# 'bagging_freq': 1,
'boost': 'gbdt',
# 'feature_fraction': 1,
'learning_rate': 0.05,
'metric':'l2',
#     'linear_tree' : True,
'num_leaves': 32,
        'verbosity':-1,
'objective': 'l2',
    'lambda_l2':10,
    'reg_sqrt':True
}

In [59]:
train_cols_view = ['ctr', 'vc_authors', 'vc_tags', 'feat_days', 'hours', 'weekday', 'day', 'vc_header', 'len_clean_title',
                   'len_clean_title_d', 'len_full_text', 'len_authors', 'len_tags', 'len_photo', 'len_overview',
                   'all_len', 'len_numbers_title', 'len_dots_title', 'len_english_text',
                   'len_numbers_text', 'len_dots_text', 'len_sites_split', 'len_diff1',  'len_parse',
                   'len_parse', 'abzac_full_text', 'abzac_full_text0', 'vc_category', 'vc_dates', 'ctr_mean',
                   'ctr_diff', 'abzac_full_text2', 'abzac_full_text3', 'dum1','new1', 'new12', 'new10',
                  'len_diff2','len_full_text_d',  'len_diff3', 'len_diff4',  'len_title',
                  'new2',  'new5']

train_cols_view += [x for x in data.columns if 'tfidf_cat_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_tags_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_aut_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_head_' in x] 
# train_cols_view += [x for x in data.columns if 'pca_cat_' in x]
train_cols_view += [x for x in data.columns if 'pca_textd_' in x]
# train_cols_view += [x for x in data.columns if 'pca_text_' in x]
bst_list, pred_train_d1, pred_depth1, score_1, fi = lgb_train(data[train_cols_view], data['depth'], ltr, split_list_new, param_lgb, 50)

C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's l2: 0.000653855 + 0.000139511
[200]	cv_agg's l2: 0.000639313 + 0.000138085
[300]	cv_agg's l2: 0.000639525 + 0.000137279
[400]	cv_agg's l2: 0.000641182 + 0.000136494
[500]	cv_agg's l2: 0.000642753 + 0.000136417
0.0006391133782469654
1 0.7957060139649272

2 0.7950799726131901

3 0.7777785005928228

4 0.7942461786400736

5 0.7982651910484987

6 0.8040054390772754

7 0.8089755372493199

8 0.8109735258974926

9 0.8138139181697598

10 0.8125937549613189



In [60]:
train_cols_view = ['ctr', 'vc_authors', 'vc_tags', 'feat_days', 'hours', 'weekday', 'day', 'vc_header', 'len_clean_title',
                   'len_clean_title_d', 'len_full_text', 'len_authors', 'len_tags', 'len_photo', 'len_overview',
                   'all_len', 'len_numbers_title', 'len_dots_title', 'len_english_text',
                   'len_numbers_text', 'len_dots_text', 'len_sites_split', 'len_diff1',  'len_parse',
                   'len_parse', 'abzac_full_text', 'abzac_full_text0', 'vc_category', 'vc_dates', 'ctr_mean',
                   'ctr_diff', 'abzac_full_text2', 'abzac_full_text3', 'dum1',
                  'len_diff2','len_full_text_d',  'len_diff3', 'len_diff4', 'len_diff5',  'len_title']

train_cols_view += [x for x in data.columns if 'tfidf_cat_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_tags_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_aut_' in x] 
train_cols_view += [x for x in data.columns if 'tfidf_head_' in x] 
# train_cols_view += [x for x in data.columns if 'pca_cat_' in x]
train_cols_view += [x for x in data.columns if 'pca_textd_' in x]
# train_cols_view += [x for x in data.columns if 'pca_text_' in x]
bst_list, pred_train_d2, pred_depth2, score_1, fi = lgb_train(data[train_cols_view], data['depth'], ltr, split_list_new, param_lgb, 50)

C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:577: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Sergei\anaconda3\lib\site-packages\lightgbm\engine.py:620: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	cv_agg's l2: 0.000658246 + 0.000140839
[200]	cv_agg's l2: 0.000649541 + 0.000140554
[300]	cv_agg's l2: 0.00065099 + 0.000141719
[400]	cv_agg's l2: 0.000652492 + 0.000144515
[500]	cv_agg's l2: 0.000654964 + 0.000146384
0.0006491699306970062
1 0.8047226970475

2 0.8032590840740452

3 0.7791135814873197

4 0.7935935503175856

5 0.7967057154230052

6 0.8017236808025899

7 0.8063440479430289

8 0.808539154648834

9 0.8108804174361303

10 0.8096182346231394



In [61]:
comp_ind = sorted(set(range(7000)) - set(have_indexs))
data['pred'] =( pred_train_d1['0'] + pred_train_d2['0']) /2
data.loc[(data.ctr > 0) , 'pred'] *= 1.
# mean_squared_error(data['full_reads_percent'][comp_ind], data['pred'][comp_ind]) / 1e9
r2_score(data['depth'][comp_ind].tolist() + data['depth'][have_indexs[have_indexs < 7000]].tolist(),
        data['pred'][comp_ind].tolist() +  data['depth'][have_indexs[have_indexs < 7000]].tolist()) 

0.8516634825052118

In [62]:
tmp = data['link'] + '_' + data['ctr'].astype('str')
tmp_vc = tmp.value_counts()
have_indexs = tmp[tmp.isin(tmp_vc[tmp_vc >= 2].index)].loc[7000:].index

In [63]:
true_dict = {}
for ind in have_indexs:
    if ind  != 8380:
        ans = data[tmp == tmp[ind]].loc[:7000-1, :][['ctr', 'dates', 'views', 'depth', 'full_reads_percent']]
        if len(ans):
            true_dict[ind] = ans.iloc[0, :].tolist()[-3:]

In [64]:
len(true_dict)

31

In [66]:
submit = pd.DataFrame()
submit['document_id'] = test['document_id']
submit['views'] = (pred_views1.mean(1) + pred_views2.mean(1)) / 2
submit['depth'] = (pred_depth1.mean(1) + pred_depth2.mean(1)) / 2
submit['full_reads_percent'] = (pred_ftr1.mean(1) + pred_ftr2.mean(1)) / 2
submit['views'] = submit['views'] * 1.1

submit['views'] = [x if i + 7000 not in true_dict else true_dict[i + 7000][0] for i, x in enumerate(submit['views']) ]
submit['depth'] = [x if i + 7000 not in true_dict else true_dict[i + 7000][1] for i, x in enumerate(submit['depth']) ]
submit['full_reads_percent'] = [x if i - 7000 not in true_dict else true_dict[i + 7000][2] for i, x in enumerate(submit['full_reads_percent']) ]

submit.to_csv('final.csv', index = None)